In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
from datetime import datetime
import datetime
import operator 
%matplotlib inline

In [32]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
denver_crime = pd.read_csv('C:\\Users\\tysau\\OneDrive\\Desktop\\crime.csv')
offense_codes = pd.read_csv('C:\\Users\\tysau\\OneDrive\\Desktop\\offense_codes.csv')

The Data for Denver Crime statistcs is relatively dense with over 400,000 records dating from 2015. So, in the following cells we will eliminate the data from the previous years to bring the data in line with the existing COVID-19 data. 

In [33]:
denver_crime.FIRST_OCCURRENCE_DATE = pd.to_datetime(denver_crime.FIRST_OCCURRENCE_DATE)
denver_crime["YEAR"] = denver_crime.FIRST_OCCURRENCE_DATE.dt.year
#df["DAY"] = df.FIRST_OCCURRENCE_DATE.dt.day
#df["DAY_OF_WEEK"] = df.FIRST_OCCURRENCE_DATE.dt.dayofweek
denver_crime["MONTH"] = denver_crime.FIRST_OCCURRENCE_DATE.dt.month
#df["HOUR"] = df.FIRST_OCCURRENCE_DATE.dt.hour

In [34]:
denver_crime = denver_crime[denver_crime.YEAR != 2018]
denver_crime = denver_crime[denver_crime.YEAR != 2017]
denver_crime = denver_crime[denver_crime.YEAR != 2016]
denver_crime = denver_crime[denver_crime.YEAR != 2015]
denver_crime = denver_crime[denver_crime.YEAR != 2019]

In [35]:
colorado_confirmed_cases = confirmed_df.loc[confirmed_df['Province_State'] == 'Colorado']
colorado_deaths = deaths_df.loc[deaths_df['Province_State'] == 'Colorado']

We will first check the missing values in the dataset and check whether any imputation is required

In [36]:
print(colorado_confirmed_cases.isnull().sum().sum())
print(colorado_deaths.isnull().sum().sum())
print(denver_crime.isnull().sum().sum())
print(offense_codes.isnull().sum().sum())

0
0
17682
0


So now we know that there are several instances where the crime dataset has empty values. These NaN values may cause problems down the road when we perform our analysis.

In [37]:
print(denver_crime.isnull().sum())

INCIDENT_ID                   0
OFFENSE_ID                    0
OFFENSE_CODE                  0
OFFENSE_CODE_EXTENSION        0
OFFENSE_TYPE_ID               0
OFFENSE_CATEGORY_ID           0
FIRST_OCCURRENCE_DATE         0
LAST_OCCURRENCE_DATE      15313
REPORTED_DATE                 0
INCIDENT_ADDRESS           1705
GEO_X                       166
GEO_Y                       166
GEO_LON                     166
GEO_LAT                     166
DISTRICT_ID                   0
PRECINCT_ID                   0
NEIGHBORHOOD_ID               0
IS_CRIME                      0
IS_TRAFFIC                    0
YEAR                          0
MONTH                         0
dtype: int64


So when we analyse the crime DF columns for null, we see that in many crimes, the last occurence date, address, and latitude, longitudes are missing in several instances. Now, the columns that have null values will not be necessary for out analysis as the coordinates, and when the crime was reported last is not necessary for our inferences. We are more concerned for when the crime is first reported.

Also, adding average or median values imputation in columns like date and latitude/logitude will lead to muddy data as it may lead to thousands of entries on the same date(average of all dates) or at the same locaton(average latitude).

Another point: The neighborhood code can cover the location of crime within the city so the insights will not be lost. 

So, before applying Tukey's rule for outlier detection, we can remove these columns, as removing 15k rows with LAST_OCCURRENCE_DATE null will lead to a loss in insights.   

In [38]:
denver_crime=denver_crime.drop(['GEO_X', 'GEO_Y', 'GEO_LON', 'GEO_LAT','LAST_OCCURRENCE_DATE','INCIDENT_ADDRESS'], axis=1)

In [29]:
denver_crime

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,REPORTED_DATE,GEO_Y,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,YEAR,MONTH
20,2020131877,2020131877570700,5707,0,criminal-trespassing,all-other-crimes,2020-02-29 20:15:00,2/29/2020 9:16:00 PM,1700184.0,6,611,five-points,1,0,2020,2
71,2020121279,2020121279544100,5441,0,traffic-accident,traffic-accident,2020-02-25 11:00:00,2/25/2020 11:02:00 AM,1691372.0,6,623,capitol-hill,0,1,2020,2
99,202029783,202029783230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,2020-01-13 23:00:00,1/14/2020 11:07:00 AM,1672896.0,4,422,college-view-south-platte,1,0,2020,1
135,202043098,202043098521201,5212,1,weapon-by-prev-offender-powpo,all-other-crimes,2020-01-20 02:02:00,1/20/2020 3:36:00 AM,1699548.0,6,611,five-points,1,0,2020,1
156,202046559,202046559230300,2303,0,theft-shoplift,larceny,2020-01-21 15:43:00,1/21/2020 5:13:00 PM,1711073.0,5,511,stapleton,1,0,2020,1
174,2020109277,2020109277131500,1315,0,aggravated-assault,aggravated-assault,2020-02-19 20:06:00,2/19/2020 11:01:00 PM,1715492.0,5,522,gateway-green-valley-ranch,1,0,2020,2
189,202045909,202045909131302,1313,2,assault-dv,other-crimes-against-persons,2020-01-21 11:30:00,1/21/2020 11:49:00 AM,1671719.0,4,422,college-view-south-platte,1,0,2020,1
192,20206000644,20206000644230400,2304,0,theft-parts-from-vehicle,theft-from-motor-vehicle,2020-01-03 14:15:00,1/17/2020 2:32:00 PM,1709856.0,2,212,elyria-swansea,1,0,2020,1
212,202075913,202075913240400,2404,0,theft-of-motor-vehicle,auto-theft,2020-02-02 12:30:00,2/3/2020 5:56:00 PM,1697347.0,6,621,north-capitol-hill,1,0,2020,2
227,2020115561,2020115561540100,5401,0,traffic-accident-hit-and-run,traffic-accident,2020-02-22 16:42:00,2/22/2020 7:23:00 PM,1694849.0,6,611,civic-center,0,1,2020,2


In [39]:
print(denver_crime.isnull().sum())

INCIDENT_ID               0
OFFENSE_ID                0
OFFENSE_CODE              0
OFFENSE_CODE_EXTENSION    0
OFFENSE_TYPE_ID           0
OFFENSE_CATEGORY_ID       0
FIRST_OCCURRENCE_DATE     0
REPORTED_DATE             0
DISTRICT_ID               0
PRECINCT_ID               0
NEIGHBORHOOD_ID           0
IS_CRIME                  0
IS_TRAFFIC                0
YEAR                      0
MONTH                     0
dtype: int64


Now we have eliminated all null values in the crime table

The following is a method to perform the outlier detection

In [57]:
# provides upper and lower limits for a column above which the value will be an outlier 
from statistics import median
def outlier_limits(values, multiplier):
    values = sorted(values, key=float)

    if len(values) >= 2:
        midpoint = int(round(len(values) / 2.0))
        q1 = median(values[:midpoint])
        q3 = median(values[midpoint:])
        iqr = q3 - q1
        lower = q1 - (iqr * multiplier)
        upper = q3 + (iqr * multiplier)
    elif values:
        lower = upper = values[0]
    else:
        lower = upper = 0
    return[lower, upper]

In [49]:
#extracts only the numerical data from the datasets 
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

denver_crime_numeric_only = denver_crime.select_dtypes(include=numerics)
colorado_deaths_numeric_only = colorado_deaths.select_dtypes(include=numerics)
colorado_confirmed_cases_numeric_only = colorado_confirmed_cases.select_dtypes(include=numerics)

In [62]:
# This method calculates the range of tolerance, above and below which lie the outliers, for each column
def calculate_range(df):
    limits = list()
    for column in df:
        temp = list()
        [low, high] = outlier_limits(df[column], 2.2)
        temp.append(low)
        temp.append(high)
        limits.append(temp)
    limits_df = pd.DataFrame(limits, columns=['lower threshold', 'upper threshold'])   
    return [limits, limits_df]

In [89]:
#data for confirmed cases
col_names = list() 
# finding the threshold ranges for each column in the colorado confirmed cases column
[limits, limits_df] = calculate_range(colorado_confirmed_cases_numeric_only)
# storing the names of each column in a list, so that this can be used to calculate the -
# - outliers in each column 
for column in colorado_confirmed_cases_numeric_only.columns: 
    col_names.append(column)

for i in range(0, len(col_names)):
    column_name = col_names[i]
    column_data = colorado_confirmed_cases_numeric_only[[column_name]]
    limits_for_column = limits[i]
    for j in range(0,len(column_data)):
        if ((column_data.iloc[j][column_name] < limits_for_column[0]) or (column_data.iloc[j][column_name] > limits_for_column[1])):
            print('Outlier in column ',column_name,' at row number:',j+1,'with the value',column_data.iloc[j][column_name] ) 

Outlier in column  UID  at row number: 65 with the value 84080008
Outlier in column  UID  at row number: 66 with the value 84090008
Outlier in column  FIPS  at row number: 65 with the value 80008.0
Outlier in column  FIPS  at row number: 66 with the value 90008.0
Outlier in column  Lat  at row number: 65 with the value 0.0
Outlier in column  Lat  at row number: 66 with the value 0.0
Outlier in column  Long_  at row number: 65 with the value 0.0
Outlier in column  Long_  at row number: 66 with the value 0.0
Outlier in column  3/6/20  at row number: 17 with the value 2
Outlier in column  3/6/20  at row number: 60 with the value 1
Outlier in column  3/7/20  at row number: 17 with the value 2
Outlier in column  3/7/20  at row number: 19 with the value 3
Outlier in column  3/7/20  at row number: 22 with the value 1
Outlier in column  3/7/20  at row number: 60 with the value 1
Outlier in column  3/8/20  at row number: 17 with the value 2
Outlier in column  3/8/20  at row number: 19 with the 

Outlier in column  3/25/20  at row number: 50 with the value 18
Outlier in column  3/25/20  at row number: 63 with the value 87
Outlier in column  3/25/20  at row number: 66 with the value 18
Outlier in column  3/26/20  at row number: 1 with the value 50
Outlier in column  3/26/20  at row number: 3 with the value 119
Outlier in column  3/26/20  at row number: 7 with the value 66
Outlier in column  3/26/20  at row number: 17 with the value 262
Outlier in column  3/26/20  at row number: 19 with the value 67
Outlier in column  3/26/20  at row number: 20 with the value 147
Outlier in column  3/26/20  at row number: 22 with the value 137
Outlier in column  3/26/20  at row number: 27 with the value 57
Outlier in column  3/26/20  at row number: 31 with the value 131
Outlier in column  3/26/20  at row number: 36 with the value 61
Outlier in column  3/26/20  at row number: 63 with the value 107
Outlier in column  3/26/20  at row number: 66 with the value 102
Outlier in column  3/27/20  at row n

Outlier in column  4/12/20  at row number: 1 with the value 647
Outlier in column  4/12/20  at row number: 3 with the value 1083
Outlier in column  4/12/20  at row number: 7 with the value 264
Outlier in column  4/12/20  at row number: 17 with the value 1247
Outlier in column  4/12/20  at row number: 19 with the value 298
Outlier in column  4/12/20  at row number: 20 with the value 452
Outlier in column  4/12/20  at row number: 22 with the value 613
Outlier in column  4/12/20  at row number: 31 with the value 763
Outlier in column  4/12/20  at row number: 36 with the value 187
Outlier in column  4/12/20  at row number: 63 with the value 738
Outlier in column  4/13/20  at row number: 1 with the value 693
Outlier in column  4/13/20  at row number: 3 with the value 1142
Outlier in column  4/13/20  at row number: 7 with the value 284
Outlier in column  4/13/20  at row number: 17 with the value 1346
Outlier in column  4/13/20  at row number: 19 with the value 304
Outlier in column  4/13/20 

Outlier in column  5/5/20  at row number: 31 with the value 1466
Outlier in column  5/5/20  at row number: 63 with the value 1955
Outlier in column  5/6/20  at row number: 1 with the value 1952
Outlier in column  5/6/20  at row number: 3 with the value 2977
Outlier in column  5/6/20  at row number: 7 with the value 700
Outlier in column  5/6/20  at row number: 17 with the value 3674
Outlier in column  5/6/20  at row number: 19 with the value 556
Outlier in column  5/6/20  at row number: 20 with the value 552
Outlier in column  5/6/20  at row number: 22 with the value 1055
Outlier in column  5/6/20  at row number: 31 with the value 1507
Outlier in column  5/6/20  at row number: 63 with the value 1988
Outlier in column  5/7/20  at row number: 1 with the value 2046
Outlier in column  5/7/20  at row number: 3 with the value 3089
Outlier in column  5/7/20  at row number: 7 with the value 720
Outlier in column  5/7/20  at row number: 17 with the value 3799
Outlier in column  5/7/20  at row n